In [1]:
import numpy as np

In [2]:
polarity_donald = np.load("Datasets/Validation/US/hashtag_donaldtrump_polarity.npy")
subjectivity_donald = np.load("Datasets/Validation/US/hashtag_donaldtrump_subjectivity.npy")
bnb_donald = np.load("Datasets/Validation/US/hashtag_donaldtrump_sentiment_BNB.npy")

In [3]:
polarity_biden = np.load("Datasets/Validation/US/hashtag_joebiden_polarity.npy")
subjectivity_biden = np.load("Datasets/Validation/US/hashtag_joebiden_subjectivity.npy")
bnb_biden = np.load("Datasets/Validation/US/hashtag_joebiden_sentiment_BNB.npy")

In [4]:
import pandas as pd

In [18]:
df_donald = pd.read_csv("Datasets/Validation/US/hashtag_donaldtrump.csv",lineterminator='\n')
df_donald["vader"] = polarity_donald
df_donald["model"] = bnb_donald
df_donald["opinion"] = subjectivity_donald

In [19]:
df_biden = pd.read_csv("Datasets/Validation/US/hashtag_joebiden.csv",lineterminator='\n')
df_biden["vader"] = polarity_biden
df_biden["model"] = bnb_biden
df_biden["opinion"] = subjectivity_biden

In [20]:
print(list(df_donald.keys()))
print(df_donald["country"])

import time

['created_at', 'tweet_id', 'tweet', 'likes', 'retweet_count', 'source', 'user_id', 'user_name', 'user_screen_name', 'user_description', 'user_join_date', 'user_followers_count', 'user_location', 'lat', 'long', 'city', 'country', 'continent', 'state', 'state_code', 'collected_at', 'vader', 'model', 'opinion']
0         United States of America
1                              NaN
2         United States of America
3                              NaN
4         United States of America
                    ...           
970914                         NaN
970915                         NaN
970916                         NaN
970917                         NaN
970918                         NaN
Name: country, Length: 970919, dtype: object


In [21]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New York', 'New Mexico', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
stateCodes = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
stateMapping = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 
                  'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NY': 'New York', 'NM': 'New Mexico', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT':  'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV':  'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'}

In [23]:
def filter(df_donald):

    df_donald['country']=df_donald['country'].replace({"United States of America" : "US", "United States" : "US"})
    df_donald = df_donald.loc[df_donald['country'] == "US"] 
    df_filtered = df_donald
    print(len(df_donald.index))
    print(len(df_filtered.index))
    print(df_donald.iterrows())
    i = 0
    
    # loop = tqdm(len(df_donald.index),leave=True)
    for index, row in df_donald.iterrows():
        i+=1
        flag = 0
        t1 = time.time()
        if row.state in states:
            flag = 2
            t2 = time.time()
            print(t2-t1)
            print("state")
        elif row.state_code in stateCodes:
            row['state'] = stateMapping[row.state_code]
            flag = 1
            t2 = time.time()
            print(t2-t1)
            print("state code")
        elif (isinstance(row.user_location,float)==False):
            words = row.user_location.split(',')
            for word in words:
                check = word.strip()
                if check in states:
                    row['state'] = check
                    flag = 1
                    break
                elif check in stateCodes:
                    row['state'] = stateMapping[check]
                    flag = 1
                    break
            t2 = time.time()
            print(t2-t1)
            print("location")
        if (flag==1):
            df_filtered.loc[index, 'state'] = row['state']
            print(row['state'])
            t2 = time.time()
            print(t2-t1)
            print("update state")
        elif (flag == 0):
            df_filtered = df_filtered.drop(index = index)
            print(row.country)
            t2 = time.time()
            print(t2-t1)
            print("drop")
        # loop.update(1)
        print(i)

In [24]:
    df_donald = filter(df_donald)
    print("Donald Done")
    df_biden = filter(df_biden)
    print("Biden Done")

213263
213263
<generator object DataFrame.iterrows at 0x7fc2f642a7b0>
6.818771362304688e-05
state
1
9.059906005859375e-06
state
2
2.7894973754882812e-05
location
US
0.04941082000732422
drop
3
8.821487426757812e-06
state
4
8.106231689453125e-06
state
5
5.9604644775390625e-06
state
6
7.867813110351562e-06
state
7
5.0067901611328125e-06
state
8
5.9604644775390625e-06
state
9
2.574920654296875e-05
location
US
0.07261776924133301
drop
10
1.71661376953125e-05
state
11
8.106231689453125e-06
state
12
6.198883056640625e-06
state
13
5.9604644775390625e-06
state
14
2.8133392333984375e-05
location
US
0.055741310119628906
drop
15
1.4066696166992188e-05
state
16
6.9141387939453125e-06
state
17
5.9604644775390625e-06
state
18
6.198883056640625e-06
state
19
5.245208740234375e-06
state
20
5.793571472167969e-05
location
Washington
0.005880117416381836
update state
21
1.2159347534179688e-05
state
22
1.2159347534179688e-05
state
23
6.198883056640625e-06
state
24
5.245208740234375e-06
state
25
5.0067901611

KeyboardInterrupt: 

In [ ]:

df_donald = pd.read_csv("Datasets/Validation/US/hashtag_donaldtrump_filtered.csv",lineterminator='\n')
df_biden = pd.read_csv("Datasets/Validation/US/hashtag_joebiden_filtered.csv",lineterminator='\n')

FileNotFoundError: [Errno 2] No such file or directory: 'Datasets/Validation/US/hashtag_donaldtrump_filtered.csv'

In [ ]:
tweets_location_plan1 = pd.DataFrame({'State': [state for state in states],
        'Trump Positive': [0 for state in states],
        'Trump Negative': [0 for state in states],
        'Trump Neutral': [0 for state in states],
        'Trump Total': [0 for state in states],
        'Biden Positive': [0 for state in states],
        'Biden Negative': [0 for state in states],
        'Biden Neutral': [0 for state in states],
        'Biden Total': [0 for state in states]})

tweets_location_plan2 = pd.DataFrame({'State': [state for state in states],
        'Trump Positive': [0 for state in states],
        'Trump Negative': [0 for state in states],
        'Trump Neutral': [0 for state in states],
        'Trump Total': [0 for state in states],
        'Biden Positive': [0 for state in states],
        'Biden Negative': [0 for state in states],
        'Biden Neutral': [0 for state in states],
        'Biden Total': [0 for state in states]})

tweets_location_plan3 = pd.DataFrame({'State': [state for state in states],
        'Trump Positive': [0 for state in states],
        'Trump Negative': [0 for state in states],
        'Trump Neutral': [0 for state in states],
        'Trump Total': [0 for state in states],
        'Biden Positive': [0 for state in states],
        'Biden Negative': [0 for state in states],
        'Biden Neutral': [0 for state in states],
        'Biden Total': [0 for state in states]})

tweets_location_plan4 = pd.DataFrame({'State': [state for state in states],
        'Trump Positive': [0 for state in states],
        'Trump Negative': [0 for state in states],
        'Trump Neutral': [0 for state in states],
        'Trump Total': [0 for state in states],
        'Biden Positive': [0 for state in states],
        'Biden Negative': [0 for state in states],
        'Biden Neutral': [0 for state in states],
        'Biden Total': [0 for state in states]})

In [ ]:
# Ideas :
    # Plan 1 : Direct count using trained model
    # Plan 2 : Direct count using VADER
    # Plan 3 : Weighted using trained and subjectivity
    # Plan 4 : Weighted using VADER and subjectivity

In [ ]:
for ind in df.index:
    print(df["vader"][ind])